In [ ]:
import pandas as pd

In [ ]:
resp = pd.read_csv('/content/amostra_recurso.csv')

In [ ]:
resp.head()

,recurso,tema,tese,num_tema_cadastrado,sugerido_1,sugerido_2,sugerido_3,sugerido_4,sugerido_5,sugerido_6,probabilidade_1,probabilidade_2,probabilidade_3,probabilidade_4,probabilidade_5,probabilidade_6,sugestao_adotada
0,EXCELENTÍSSIMO SENHOR VICE-PRESIDENTE DO EGRÉG...,Definir se a liquidação prévia do julgado é re...,NaN,1169,1056,1130.0,1178.0,1033.0,1086.0,NaN,37,24,13,13,13,0,0
1,\r\nADVOCACIA-GERAL DA UNIÃO\r\nPROCURADORIA-G...,Fornecimento de Equipamento de Proteção Indivi...,I - O direito à aposentadoria especial pressup...,555,1083,694.0,534.0,555.0,NaN,NaN,35,30,31,4,0,0,4
2,\r\nADVOCACIA-GERAL DA UNIÃO\r\nPROCURADORIA-G...,Definir se é possível ao magistrado fixar crit...,NaN,7,1090,534.0,1081.0,1083.0,694.0,NaN,33,23,22,11,11,0,0
3,PROCURADORIA-GERAL DA FAZENDA NACIONAL\r\n\r\n...,Inclusão da COFINS e da contribuição ao PIS em...,NaN,1067,1125,1182.0,NaN,NaN,NaN,NaN,87,13,0,0,0,0,0
4,ADVOCACIA-GERAL DA UNIÃƒOPROCURADORIA-GERAL FE...,Definir se a demanda previdenciária cujo valor...,NaN,1081,1081,NaN,NaN,NaN,NaN,NaN,100,0,0,0,0,0,1


**Modelo Legal-BERTimbau**
[huggingface](https://huggingface.co/rufimelo/Legal-BERTimbau-base)

@inproceedings{souza2020bertimbau,
  author    = {F{\'a}bio Souza and
               Rodrigo Nogueira and
               Roberto Lotufo},
  title     = {{BERT}imbau: pretrained {BERT} models for {B}razilian {P}ortuguese},
  booktitle = {9th Brazilian Conference on Intelligent Systems, {BRACIS}, Rio Grande do Sul, Brazil, October 20-23 (to appear)},
  year      = {2020}
}


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.2 MB/s eta 0:00:00


In [ ]:
import  torch
from  transformers  import  AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("rufimelo/Legal-BERTimbau-base")
model = AutoModel.from_pretrained('rufimelo/Legal-BERTimbau-base')

def ler_recurso(dataframe, linha, coluna):
    texto = dataframe.at[linha, coluna]
    return texto

conteudo = ler_recurso(resp,1,'recurso')
input_ids = tokenizer.encode(conteudo, return_tensors='pt')
#Limitacao de tamanho do texto para usar o modelo , para teste foi limitada a dimensao do tensor em 512
input_ids512 = input_ids[:, :512]

with  torch.no_grad():
    outs = model(input_ids512)
    encoded = outs[0][0, 1:-1]



Some weights of BertModel were not initialized from the model checkpoint at rufimelo/Legal-BERTimbau-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(encoded)

tensor([[ 0.3504, -0.6196,  0.8408,  ..., -0.6311, -0.0492, -0.2174],
        [-0.1156, -0.3521,  0.5572,  ..., -0.4350, -0.4658, -0.0934],
        [-0.1086, -0.3734,  0.2686,  ...,  0.3604,  0.0313, -0.3450],
        ...,
        [ 0.0545, -0.3944,  0.3708,  ..., -0.8043, -0.4407,  0.5829],
        [ 0.3855, -0.1784, -0.4938,  ..., -0.1290, -0.0845,  0.1714],
        [ 0.1156,  0.0647, -0.5313,  ..., -0.6083,  0.1171, -0.4277]])


In [ ]:
print("Shape:", encoded.shape)
print("Número de dimensões:", encoded.dim())
print("Tipo de dado:", encoded.dtype)
print("Dispositivo (CPU/GPU):", encoded.device)
print("Número total de elementos:", encoded.numel())

Shape: torch.Size([8, 768])
Número de dimensões: 2
Tipo de dado: torch.float32
Dispositivo (CPU/GPU): cpu
Número total de elementos: 6144


# **Implementação Doc2Vec com biblioteca gensim**

In [1]:
import os
import gensim

test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
lee_train_file = os.path.join(test_data_dir, 'lee_background.cor')
lee_test_file = os.path.join(test_data_dir, 'lee.cor')

### **Pré-processamento**

In [2]:
import smart_open

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

In [3]:
print(train_corpus[:2])

[TaggedDocument(words=['hundreds', 'of', 'people', 'have', 'been', 'forced', 'to', 'vacate', 'their', 'homes', 'in', 'the', 'southern', 'highlands', 'of', 'new', 'south', 'wales', 'as', 'strong', 'winds', 'today', 'pushed', 'huge', 'bushfire', 'towards', 'the', 'town', 'of', 'hill', 'top', 'new', 'blaze', 'near', 'goulburn', 'south', 'west', 'of', 'sydney', 'has', 'forced', 'the', 'closure', 'of', 'the', 'hume', 'highway', 'at', 'about', 'pm', 'aedt', 'marked', 'deterioration', 'in', 'the', 'weather', 'as', 'storm', 'cell', 'moved', 'east', 'across', 'the', 'blue', 'mountains', 'forced', 'authorities', 'to', 'make', 'decision', 'to', 'evacuate', 'people', 'from', 'homes', 'in', 'outlying', 'streets', 'at', 'hill', 'top', 'in', 'the', 'new', 'south', 'wales', 'southern', 'highlands', 'an', 'estimated', 'residents', 'have', 'left', 'their', 'homes', 'for', 'nearby', 'mittagong', 'the', 'new', 'south', 'wales', 'rural', 'fire', 'service', 'says', 'the', 'weather', 'conditions', 'which', '

In [4]:
print(test_corpus[:2])

[['the', 'national', 'executive', 'of', 'the', 'strife', 'torn', 'democrats', 'last', 'night', 'appointed', 'little', 'known', 'west', 'australian', 'senator', 'brian', 'greig', 'as', 'interim', 'leader', 'shock', 'move', 'likely', 'to', 'provoke', 'further', 'conflict', 'between', 'the', 'party', 'senators', 'and', 'its', 'organisation', 'in', 'move', 'to', 'reassert', 'control', 'over', 'the', 'party', 'seven', 'senators', 'the', 'national', 'executive', 'last', 'night', 'rejected', 'aden', 'ridgeway', 'bid', 'to', 'become', 'interim', 'leader', 'in', 'favour', 'of', 'senator', 'greig', 'supporter', 'of', 'deposed', 'leader', 'natasha', 'stott', 'despoja', 'and', 'an', 'outspoken', 'gay', 'rights', 'activist'], ['cash', 'strapped', 'financial', 'services', 'group', 'amp', 'has', 'shelved', 'million', 'plan', 'to', 'buy', 'shares', 'back', 'from', 'investors', 'and', 'will', 'raise', 'million', 'in', 'fresh', 'capital', 'after', 'profits', 'crashed', 'in', 'the', 'six', 'months', 'to'

In [5]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [6]:
model.build_vocab(train_corpus)

In [7]:
print(f"Word 'penalty' appeared {model.wv.get_vecattr('penalty', 'count')} times in the training corpus.")

Word 'penalty' appeared 4 times in the training corpus.


In [8]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

### **Usando modelo treinado para criar embedding de 50 dimensões a partir de um documento**

In [9]:
vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
print(vector)

[-0.09489255 -0.3239154  -0.1007698   0.20849843  0.08287817 -0.06486095
 -0.0527963   0.01311693 -0.28106943 -0.10581531  0.19818015 -0.00569113
 -0.06821475 -0.03600449 -0.1336134  -0.23196214  0.14489928  0.25862664
  0.15737635 -0.08721808  0.0837669  -0.02155922  0.08978836  0.03663864
  0.00091677 -0.11039349 -0.14523797 -0.02727995 -0.14722757 -0.09177545
  0.4151152  -0.01962528  0.1387924   0.11256336  0.2497105   0.04271441
 -0.13254812 -0.31660327 -0.04524599 -0.05006953  0.03643869 -0.07641411
  0.03496949 -0.06547639  0.10941408  0.00517405 -0.17135692 -0.07929065
  0.12492654 -0.02853565]


### ***Avaliando modelo usando dados ***

In [10]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [11]:
import collections

counter = collections.Counter(ranks)
print(counter)

Counter({0: 292, 1: 8})


In [12]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v